In [1]:
import pandas as pd
import os
import torch

In [2]:
# pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# hugging-face model 불러오기
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.3 MB/s eta 0:00:00


In [5]:
model = model.to(device)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# 코퍼스 키워드로 추출된 공시 데이터 불러오기
path = '/content/drive/MyDrive/1. preprocessing/'
file_list = os.listdir(path)
# 이중 공시 데이터만 가져옴
file_list = [file for file in file_list if file.startswith("key_bert")]

In [13]:
for file in file_list:
    df = pd.read_csv(path+file)
    acc = df.copy().drop_duplicates(subset = 'keyword')
    acc['text'] = ''

    for _, row in acc.iterrows():
        s = ''
        sub = df[df['keyword']==row['keyword']]
        for _, i in sub.iterrows():
            s += i['text']
        acc.loc[acc['keyword']==row['keyword'],'text'] = s

    out = acc.copy()
    out['text'] = ''

    for _, item in acc.iterrows():
        inputs = item['text']
        inputs = tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")
        inputs = inputs.to(device)
        output = model.generate(**inputs, num_beams=3, do_sample=True, min_length=10, max_length=128)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=False)[0]
        result = nltk.sent_tokenize(decoded_output.strip())[0]
        out.loc[out['keyword']==item['keyword'], 'text'] = result

    out['text'] = out['text'].apply(lambda x: x.replace('<pad>','').replace('</s>',''))
    
    # output 폴더 없으면 만들기 
    if not os.path.isdir(path + 'output/'):
        os.mkdir(path + 'output/')
    out.to_csv(path+'summarized_from_key_bert_'+file.replace("key_bert_", ""), index=False, encoding='utf-8-sig')
    acc.to_csv(path+'src_for_summarize'+file.replace("key_bert_", ""), index=False, encoding='utf-8-sig')